In [4]:
import open3d as o3d
import numpy as np
import copy
import random

# ───────────────────── 기본 함수 정의 ─────────────────────────────

# 포즈 파일 읽기: 각 프레임의 4x4 homogeneous transformation matrix 반환
def load_poses(pose_file):
    poses = []
    with open(pose_file, 'r') as f:
        for line in f:
            vals = np.array(list(map(float, line.split())))
            pose = np.vstack((vals.reshape(3, 4), [0, 0, 0, 1]))
            poses.append(pose)
    return poses

# 라벨 매핑: 원래 label 값을 간단한 label로 재매핑
learning_map = {
    0: 0, 1: 0, 9: 1, 10: 1, 11: 1, 13: 1, 15: 1, 16: 1, 18: 1, 20: 1,
    30: 1, 31: 1, 32: 1, 40: 1, 44: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1,
    60: 1, 70: 1, 71: 1, 72: 1, 80: 1, 81: 1, 99: 1, 
    251: 2, 252: 2, 253: 2, 254: 2, 255: 2, 256: 2, 257: 2, 258: 2, 259: 2,
}

def relabel(pcds_labels, label_map):
    result_labels = np.zeros((pcds_labels.shape[0],), dtype=pcds_labels.dtype)
    for key in label_map:
        mask = pcds_labels == key
        result_labels[mask] = label_map[key]
    return result_labels

# 2.5D Bounding Box 계산 (XY 평면에서 회전 가능, Z축은 수직)
def get_2d_obb(pcd):
    points = np.asarray(pcd.points)
    xy = points[:, :2]
    mean_xy = xy.mean(axis=0)
    xy_centered = xy - mean_xy

    # PCA (2x2)
    cov = np.cov(xy_centered.T)
    eigvals, eigvecs = np.linalg.eigh(cov)
    R_2d = eigvecs  # 2x2 회전 행렬
    rotated = xy_centered @ R_2d

    min_xy = rotated.min(axis=0)
    max_xy = rotated.max(axis=0)
    # 회전 좌표계 상의 코너들
    corners = np.array([
        [min_xy[0], min_xy[1]],
        [max_xy[0], min_xy[1]],
        [max_xy[0], max_xy[1]],
        [min_xy[0], max_xy[1]],
    ])
    # 원래 좌표계로 복원
    corners_orig = corners @ R_2d.T + mean_xy

    min_z = points[:, 2].min()
    max_z = points[:, 2].max()

    center_xy = corners_orig.mean(axis=0)
    center_z = (min_z + max_z) / 2.0
    center = np.array([center_xy[0], center_xy[1], center_z])

    extent_xy = max_xy - min_xy  # 길이, 너비 (회전 좌표계 상)
    extent_z = max_z - min_z      # 높이
    extent = np.array([extent_xy[0], extent_xy[1], extent_z])

    R_full = np.eye(3)
    R_full[:2, :2] = R_2d

    obb = o3d.geometry.OrientedBoundingBox(center, R_full, extent)
    return obb

# bbox(OrientedBoundingBox)를 transformation matrix T로 변환하는 함수
def transform_obb(obb, T):
    center_h = np.hstack((obb.center, 1))
    new_center = (T @ center_h)[:3]
    new_R = T[:3, :3] @ obb.R
    new_extent = obb.extent
    new_obb = o3d.geometry.OrientedBoundingBox(new_center, new_R, new_extent)
    new_obb.color = obb.color
    return new_obb

# 인스턴스 매칭: 기준 중심(center)과 목록 내 bbox의 중심 간 거리가 threshold보다 작으면 매칭
def find_matching_bbox(center, bbox_list, threshold=2.0):
    min_dist = threshold
    matched_bbox = None
    for bbox in bbox_list:
        dist = np.linalg.norm(center - bbox.center)
        if dist < min_dist:
            min_dist = dist
            matched_bbox = bbox
    return matched_bbox

# ───────────────────── 경로 및 기본 설정 ─────────────────────────────

seq = "08"
data_dir = "/home/workspace/KITTI/dataset/sequences/" + seq
velo_dir = data_dir + "/velodyne"
label_dir = data_dir + "/labels"
pose_path = data_dir + "/poses.txt"

# 사용할 프레임 번호: 48 (t-2), 49 (t-1), 50 (t), 51 (t+1)
frames_to_process = [48, 49, 50, 51]
ref_frame = 50  # 기준 좌표계 (t 시점)

# 프레임별 시각화를 위한 컬러 (움직이는 객체: label==2)
# 단, static(라벨 0, 1)는 검은색으로 표시
frame_color = {
    48: [1, 0, 0],  # 빨강
    49: [0, 1, 0],  # 초록
    50: [0, 0, 1],  # 파랑
}

# DBSCAN 파라미터 및 기타 설정
dbscan_eps = 0.5
dbscan_min_points = 10
distance_filter_min = 2.0
distance_filter_max = 50.0

# ───────────────────── 포즈 및 파일 읽기 ─────────────────────────────
poses = load_poses(pose_path)
T_ref = poses[ref_frame]        # 기준 프레임 (t) 포즈
T_ref_inv = np.linalg.inv(T_ref)

# ───────────────────── 프레임별 처리 ─────────────────────────────
# bbox_dict: 프레임별 bbox 리스트 저장 (좌표계 변환: 기준(ref_frame) 좌표계)
bbox_dict = {}
# 포인트 클라우드 누적 저장 (프레임 48~50)
accumulated_points = []
accumulated_colors = []

for fid in frames_to_process[:-1]:  # 프레임 48, 49, 50
    file_id = f"{fid:06d}"
    bin_path = f"{velo_dir}/{file_id}.bin"
    label_path = f"{label_dir}/{file_id}.label"
    
    # 포인트 클라우드 및 라벨 읽기
    pcd_data = np.fromfile(bin_path, dtype=np.float32).reshape((-1, 4))
    pcd_label = np.fromfile(label_path, dtype=np.uint32).reshape((-1))
    sem_label = pcd_label & 0xFFFF
    labels = relabel(sem_label, learning_map)
    
    # 포인트 필터링 (거리 범위)
    xyz = pcd_data[:, :3]
    dists = np.linalg.norm(xyz, axis=1)
    mask = (dists >= distance_filter_min) & (dists <= distance_filter_max)
    xyz = xyz[mask]
    labels = labels[mask]
    
    # 누적 포인트 클라우드 샘플링: label이 2이면 해당 프레임의 색상, static(0,1)은 항상 검은색
    if xyz.shape[0] > 0:
        sample_ratio = 0.05 
        num_samples = max(1, int(xyz.shape[0] * sample_ratio))
        sample_indices = np.random.choice(np.arange(xyz.shape[0]), num_samples, replace=False)
        sampled_xyz = xyz[sample_indices]
        sample_labels = labels[sample_indices]
        sampled_colors = np.array([ frame_color[fid] if lbl == 2 else [0, 0, 0] for lbl in sample_labels ])
        
        # 각 프레임의 포즈를 기준(ref_frame) 좌표계로 변환
        T_frame = poses[fid]
        T_transform = T_ref_inv @ T_frame
        ones = np.ones((sampled_xyz.shape[0], 1))
        sampled_xyz_h = np.hstack((sampled_xyz, ones))
        sampled_xyz_trans = (T_transform @ sampled_xyz_h.T).T[:, :3]
        
        accumulated_points.append(sampled_xyz_trans)
        accumulated_colors.append(sampled_colors)
    
    # ── DBSCAN을 통해 label이 2인(움직이는) 점만 대상으로 bbox 추출 ──
    target_mask = (labels == 2)
    bbox_list = []
    if np.sum(target_mask) > 0:
        target_xyz = xyz[target_mask]
        pcd_target = o3d.geometry.PointCloud()
        pcd_target.points = o3d.utility.Vector3dVector(target_xyz)
        clusters = np.array(pcd_target.cluster_dbscan(eps=dbscan_eps, min_points=dbscan_min_points, print_progress=False))
        for cluster_id in range(clusters.min(), clusters.max() + 1):
            cluster_mask = (clusters == cluster_id)
            cluster_points = target_xyz[cluster_mask]
            if cluster_points.shape[0] == 0:
                continue
            # 이상치 제거: 클러스터 내 각 점과 중심간의 거리가 k배 이상인 점 무시
            cluster_center = np.mean(cluster_points, axis=0)
            distances = np.linalg.norm(cluster_points - cluster_center, axis=1)
            median_distance = np.median(distances)
            k = 2.0  # 임계값 계수
            inlier_mask = distances < k * median_distance
            filtered_points = cluster_points[inlier_mask]
            if filtered_points.shape[0] < 5:  # 너무 적은 점이면 무시
                continue
            
            pcd_cluster = o3d.geometry.PointCloud()
            pcd_cluster.points = o3d.utility.Vector3dVector(filtered_points)
            obb = get_2d_obb(pcd_cluster)
            # transform: 각 프레임의 bbox를 기준(ref_frame) 좌표계로 변환
            T_frame = poses[fid]
            T_transform = T_ref_inv @ T_frame
            # bbox 컬러: 해당 프레임의 moving object 색상 (static은 제외됨)
            obb.color = frame_color[fid]
            obb_transformed = transform_obb(obb, T_transform)
            bbox_list.append(obb_transformed)
    bbox_dict[fid] = bbox_list

# ───────────────────── 프레임 51 (t+1)의 실제 bbox 계산 (보라색) ─────────────────────────────
actual_bboxes = []
fid = frames_to_process[-1]  # 51
file_id = f"{fid:06d}"
bin_path = f"{velo_dir}/{file_id}.bin"
label_path = f"{label_dir}/{file_id}.label"
pcd_data = np.fromfile(bin_path, dtype=np.float32).reshape((-1, 4))
pcd_label = np.fromfile(label_path, dtype=np.uint32).reshape((-1))
sem_label = pcd_label & 0xFFFF
labels = relabel(sem_label, learning_map)

xyz = pcd_data[:, :3]
dists = np.linalg.norm(xyz, axis=1)
mask = (dists >= distance_filter_min) & (dists <= distance_filter_max)
xyz = xyz[mask]
labels = labels[mask]

target_mask = (labels == 2)
if np.sum(target_mask) > 0:
    target_xyz = xyz[target_mask]
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_xyz)
    clusters = np.array(pcd_target.cluster_dbscan(eps=dbscan_eps, min_points=dbscan_min_points, print_progress=False))
    for cluster_id in range(clusters.min(), clusters.max() + 1):
        cluster_mask = (clusters == cluster_id)
        cluster_points = target_xyz[cluster_mask]
        if cluster_points.shape[0] == 0:
            continue
        # 이상치 제거
        cluster_center = np.mean(cluster_points, axis=0)
        distances = np.linalg.norm(cluster_points - cluster_center, axis=1)
        median_distance = np.median(distances)
        k = 2.0
        inlier_mask = distances < k * median_distance
        filtered_points = cluster_points[inlier_mask]
        if filtered_points.shape[0] < 5:
            continue
        pcd_cluster = o3d.geometry.PointCloud()
        pcd_cluster.points = o3d.utility.Vector3dVector(filtered_points)
        obb = get_2d_obb(pcd_cluster)
        T_frame = poses[fid]
        T_transform = T_ref_inv @ T_frame
        obb.color = [0.5, 0, 0.5]  # 보라색
        obb_transformed = transform_obb(obb, T_transform)
        actual_bboxes.append(obb_transformed)

# ───────────────────── t-2, t-1, t를 이용하여 t+1 예측 bbox 생성 (노란색) ─────────────────────────────
predicted_bboxes = []
threshold_match = 2.0  # 매칭을 위한 중심 간 최대 거리

# 기준 프레임 t (frame 50)의 bbox를 기준으로 매칭
for bbox_t in bbox_dict.get(ref_frame, []):
    center_t = bbox_t.center
    # 프레임 49 (t-1)에서 매칭
    bboxes_t_minus1 = bbox_dict.get(ref_frame - 1, [])
    match_t_minus1 = find_matching_bbox(center_t, bboxes_t_minus1, threshold=threshold_match)
    if match_t_minus1 is None:
        continue
    center_t_minus1 = match_t_minus1.center
    # 프레임 48 (t-2)에서 매칭
    bboxes_t_minus2 = bbox_dict.get(ref_frame - 2, [])
    match_t_minus2 = find_matching_bbox(match_t_minus1.center, bboxes_t_minus2, threshold=threshold_match)
    if match_t_minus2 is None:
        # t-2 매칭이 없으면, 단순히 t와 t-1의 차이로 속도 계산
        velocity = center_t - center_t_minus1
    else:
        center_t_minus2 = match_t_minus2.center
        velocity1 = center_t - center_t_minus1
        velocity2 = center_t_minus1 - center_t_minus2
        velocity = (velocity1 + velocity2) / 2.0
    predicted_center = center_t + velocity
    pred_bbox = copy.deepcopy(bbox_t)
    pred_bbox.center = predicted_center
    pred_bbox.color = [1, 1, 0]  # 노란색
    predicted_bboxes.append(pred_bbox)

# ───────────────────── 누적 포인트 클라우드 생성 (프레임 48~50) ─────────────────────────────
if accumulated_points:
    all_points = np.vstack(accumulated_points)
    all_colors = np.vstack(accumulated_colors)
    accumulated_pcd = o3d.geometry.PointCloud()
    accumulated_pcd.points = o3d.utility.Vector3dVector(all_points)
    accumulated_pcd.colors = o3d.utility.Vector3dVector(all_colors)
else:
    accumulated_pcd = None

# ───────────────────── 최종 시각화 ─────────────────────────────
geometries = []
if accumulated_pcd is not None:
    geometries.append(accumulated_pcd)
for bbox in actual_bboxes:
    geometries.append(bbox)
for bbox in predicted_bboxes:
    geometries.append(bbox)

o3d.visualization.draw_geometries(geometries)


In [5]:
import open3d as o3d
import numpy as np
import copy
import random

# ───────────────────── 기본 함수 정의 ─────────────────────────────

# 포즈 파일 읽기: 각 프레임의 4x4 homogeneous transformation matrix 반환
def load_poses(pose_file):
    poses = []
    with open(pose_file, 'r') as f:
        for line in f:
            vals = np.array(list(map(float, line.split())))
            pose = np.vstack((vals.reshape(3, 4), [0, 0, 0, 1]))
            poses.append(pose)
    return poses

# 라벨 매핑: 원래 label 값을 간단한 label로 재매핑
learning_map = {
    0: 0, 1: 0, 9: 1, 10: 1, 11: 1, 13: 1, 15: 1, 16: 1, 18: 1, 20: 1,
    30: 1, 31: 1, 32: 1, 40: 1, 44: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1,
    60: 1, 70: 1, 71: 1, 72: 1, 80: 1, 81: 1, 99: 1, 
    251: 2, 252: 2, 253: 2, 254: 2, 255: 2, 256: 2, 257: 2, 258: 2, 259: 2,
}

def relabel(pcds_labels, label_map):
    result_labels = np.zeros((pcds_labels.shape[0],), dtype=pcds_labels.dtype)
    for key in label_map:
        mask = pcds_labels == key
        result_labels[mask] = label_map[key]
    return result_labels

# 2.5D Bounding Box 계산 (XY 평면에서 회전 가능, Z축은 수직)
def get_2d_obb(pcd):
    points = np.asarray(pcd.points)
    xy = points[:, :2]
    mean_xy = xy.mean(axis=0)
    xy_centered = xy - mean_xy

    # PCA (2x2)
    cov = np.cov(xy_centered.T)
    eigvals, eigvecs = np.linalg.eigh(cov)
    R_2d = eigvecs  # 2x2 회전 행렬
    rotated = xy_centered @ R_2d

    min_xy = rotated.min(axis=0)
    max_xy = rotated.max(axis=0)
    # 회전 좌표계 상의 코너들
    corners = np.array([
        [min_xy[0], min_xy[1]],
        [max_xy[0], min_xy[1]],
        [max_xy[0], max_xy[1]],
        [min_xy[0], max_xy[1]],
    ])
    # 원래 좌표계로 복원
    corners_orig = corners @ R_2d.T + mean_xy

    min_z = points[:, 2].min()
    max_z = points[:, 2].max()

    center_xy = corners_orig.mean(axis=0)
    center_z = (min_z + max_z) / 2.0
    center = np.array([center_xy[0], center_xy[1], center_z])

    extent_xy = max_xy - min_xy  # 길이, 너비 (회전 좌표계 상)
    extent_z = max_z - min_z      # 높이
    extent = np.array([extent_xy[0], extent_xy[1], extent_z])

    R_full = np.eye(3)
    R_full[:2, :2] = R_2d

    obb = o3d.geometry.OrientedBoundingBox(center, R_full, extent)
    return obb

# bbox(OrientedBoundingBox)를 transformation matrix T로 변환하는 함수
def transform_obb(obb, T):
    center_h = np.hstack((obb.center, 1))
    new_center = (T @ center_h)[:3]
    new_R = T[:3, :3] @ obb.R
    new_extent = obb.extent
    new_obb = o3d.geometry.OrientedBoundingBox(new_center, new_R, new_extent)
    new_obb.color = obb.color
    return new_obb

# 인스턴스 매칭: 기준 중심(center)과 목록 내 bbox의 중심 간 거리가 threshold보다 작으면 매칭
def find_matching_bbox(center, bbox_list, threshold=2.0):
    min_dist = threshold
    matched_bbox = None
    for bbox in bbox_list:
        dist = np.linalg.norm(center - bbox.center)
        if dist < min_dist:
            min_dist = dist
            matched_bbox = bbox
    return matched_bbox

# ───────────────────── 경로 및 기본 설정 ─────────────────────────────

seq = "08"
data_dir = "/home/workspace/KITTI/dataset/sequences/" + seq
velo_dir = data_dir + "/velodyne"
label_dir = data_dir + "/labels"
pose_path = data_dir + "/poses.txt"

# 사용할 프레임 번호: 48 (t-2), 49 (t-1), 50 (t), 51 (t+1)
frames_to_process = [48, 49, 50, 51]
ref_frame = 50  # 기준 좌표계 (t 시점)

# 프레임별 시각화를 위한 컬러 (움직이는 객체: label==2)
# 단, static(라벨 0, 1)는 검은색으로 표시
frame_color = {
    48: [1, 0, 0],  # 빨강
    49: [0, 1, 0],  # 초록
    50: [0, 0, 1],  # 파랑
}

# DBSCAN 파라미터 및 기타 설정
dbscan_eps = 0.5
dbscan_min_points = 10
distance_filter_min = 2.0
distance_filter_max = 50.0

# ───────────────────── 포즈 및 파일 읽기 ─────────────────────────────
poses = load_poses(pose_path)
T_ref = poses[ref_frame]        # 기준 프레임 (t) 포즈
T_ref_inv = np.linalg.inv(T_ref)

# ───────────────────── 프레임별 처리 ─────────────────────────────
# bbox_dict: 프레임별 bbox 리스트 저장 (좌표계 변환: 기준(ref_frame) 좌표계)
bbox_dict = {}
# 포인트 클라우드 누적 저장 (프레임 48~50)
accumulated_points = []
accumulated_colors = []

for fid in frames_to_process[:-1]:  # 프레임 48, 49, 50
    file_id = f"{fid:06d}"
    bin_path = f"{velo_dir}/{file_id}.bin"
    label_path = f"{label_dir}/{file_id}.label"
    
    # 포인트 클라우드 및 라벨 읽기
    pcd_data = np.fromfile(bin_path, dtype=np.float32).reshape((-1, 4))
    pcd_label = np.fromfile(label_path, dtype=np.uint32).reshape((-1))
    sem_label = pcd_label & 0xFFFF
    labels = relabel(sem_label, learning_map)
    
    # 포인트 필터링 (거리 범위)
    xyz = pcd_data[:, :3]
    dists = np.linalg.norm(xyz, axis=1)
    mask = (dists >= distance_filter_min) & (dists <= distance_filter_max)
    xyz = xyz[mask]
    labels = labels[mask]
    
    # 누적 포인트 클라우드 샘플링: label이 2이면 해당 프레임의 색상, static(0,1)은 항상 검은색
    if xyz.shape[0] > 0:
        sample_ratio = 0.05 
        num_samples = max(1, int(xyz.shape[0] * sample_ratio))
        sample_indices = np.random.choice(np.arange(xyz.shape[0]), num_samples, replace=False)
        sampled_xyz = xyz[sample_indices]
        sample_labels = labels[sample_indices]
        sampled_colors = np.array([ frame_color[fid] if lbl == 2 else [0, 0, 0] for lbl in sample_labels ])
        
        # 각 프레임의 포즈를 기준(ref_frame) 좌표계로 변환
        T_frame = poses[fid]
        T_transform = T_ref_inv @ T_frame
        ones = np.ones((sampled_xyz.shape[0], 1))
        sampled_xyz_h = np.hstack((sampled_xyz, ones))
        sampled_xyz_trans = (T_transform @ sampled_xyz_h.T).T[:, :3]
        
        accumulated_points.append(sampled_xyz_trans)
        accumulated_colors.append(sampled_colors)
    
    # ── DBSCAN을 통해 label이 2인(움직이는) 점만 대상으로 bbox 추출 ──
    target_mask = (labels == 2)
    bbox_list = []
    if np.sum(target_mask) > 0:
        target_xyz = xyz[target_mask]
        pcd_target = o3d.geometry.PointCloud()
        pcd_target.points = o3d.utility.Vector3dVector(target_xyz)
        clusters = np.array(pcd_target.cluster_dbscan(eps=dbscan_eps, min_points=dbscan_min_points, print_progress=False))
        for cluster_id in range(clusters.min(), clusters.max() + 1):
            cluster_mask = (clusters == cluster_id)
            cluster_points = target_xyz[cluster_mask]
            if cluster_points.shape[0] == 0:
                continue
            # 이상치 제거: 클러스터 내 각 점과 중심간의 거리가 k배 이상인 점 무시
            cluster_center = np.mean(cluster_points, axis=0)
            distances = np.linalg.norm(cluster_points - cluster_center, axis=1)
            median_distance = np.median(distances)
            k = 2.0  # 임계값 계수
            inlier_mask = distances < k * median_distance
            filtered_points = cluster_points[inlier_mask]
            if filtered_points.shape[0] < 5:  # 너무 적은 점이면 무시
                continue
            
            pcd_cluster = o3d.geometry.PointCloud()
            pcd_cluster.points = o3d.utility.Vector3dVector(filtered_points)
            obb = get_2d_obb(pcd_cluster)
            # transform: 각 프레임의 bbox를 기준(ref_frame) 좌표계로 변환
            T_frame = poses[fid]
            T_transform = T_ref_inv @ T_frame
            # bbox 컬러: 해당 프레임의 moving object 색상 (static은 제외됨)
            obb.color = frame_color[fid]
            obb_transformed = transform_obb(obb, T_transform)
            bbox_list.append(obb_transformed)
    bbox_dict[fid] = bbox_list

# ───────────────────── 프레임 51 (t+1)의 실제 bbox 계산 (보라색) ─────────────────────────────
actual_bboxes = []
fid = frames_to_process[-1]  # 51
file_id = f"{fid:06d}"
bin_path = f"{velo_dir}/{file_id}.bin"
label_path = f"{label_dir}/{file_id}.label"
pcd_data = np.fromfile(bin_path, dtype=np.float32).reshape((-1, 4))
pcd_label = np.fromfile(label_path, dtype=np.uint32).reshape((-1))
sem_label = pcd_label & 0xFFFF
labels = relabel(sem_label, learning_map)

xyz = pcd_data[:, :3]
dists = np.linalg.norm(xyz, axis=1)
mask = (dists >= distance_filter_min) & (dists <= distance_filter_max)
xyz = xyz[mask]
labels = labels[mask]

target_mask = (labels == 2)
if np.sum(target_mask) > 0:
    target_xyz = xyz[target_mask]
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_xyz)
    clusters = np.array(pcd_target.cluster_dbscan(eps=dbscan_eps, min_points=dbscan_min_points, print_progress=False))
    for cluster_id in range(clusters.min(), clusters.max() + 1):
        cluster_mask = (clusters == cluster_id)
        cluster_points = target_xyz[cluster_mask]
        if cluster_points.shape[0] == 0:
            continue
        # 이상치 제거
        cluster_center = np.mean(cluster_points, axis=0)
        distances = np.linalg.norm(cluster_points - cluster_center, axis=1)
        median_distance = np.median(distances)
        k = 2.0
        inlier_mask = distances < k * median_distance
        filtered_points = cluster_points[inlier_mask]
        if filtered_points.shape[0] < 5:
            continue
        pcd_cluster = o3d.geometry.PointCloud()
        pcd_cluster.points = o3d.utility.Vector3dVector(filtered_points)
        obb = get_2d_obb(pcd_cluster)
        T_frame = poses[fid]
        T_transform = T_ref_inv @ T_frame
        obb.color = [0.5, 0, 0.5]  # 보라색
        obb_transformed = transform_obb(obb, T_transform)
        actual_bboxes.append(obb_transformed)

# ───────────────────── t-2, t-1, t를 이용하여 t+1 예측 bbox 생성 (노란색) ─────────────────────────────
predicted_bboxes = []
threshold_match = 2.0  # 매칭을 위한 중심 간 최대 거리

# 기준 프레임 t (frame 50)의 bbox를 기준으로 매칭
for bbox_t in bbox_dict.get(ref_frame, []):
    center_t = bbox_t.center
    # 프레임 49 (t-1)에서 매칭
    bboxes_t_minus1 = bbox_dict.get(ref_frame - 1, [])
    match_t_minus1 = find_matching_bbox(center_t, bboxes_t_minus1, threshold=threshold_match)
    if match_t_minus1 is None:
        continue
    center_t_minus1 = match_t_minus1.center
    # 프레임 48 (t-2)에서 매칭
    bboxes_t_minus2 = bbox_dict.get(ref_frame - 2, [])
    match_t_minus2 = find_matching_bbox(match_t_minus1.center, bboxes_t_minus2, threshold=threshold_match)
    if match_t_minus2 is None:
        # t-2 매칭이 없으면, 단순히 t와 t-1의 차이로 속도 계산
        velocity = center_t - center_t_minus1
    else:
        center_t_minus2 = match_t_minus2.center
        velocity1 = center_t - center_t_minus1
        velocity2 = center_t_minus1 - center_t_minus2
        velocity = (velocity1 + velocity2) / 2.0
    predicted_center = center_t + velocity
    pred_bbox = copy.deepcopy(bbox_t)
    pred_bbox.center = predicted_center
    pred_bbox.color = [1, 1, 0]  # 노란색
    predicted_bboxes.append(pred_bbox)

# ───────────────────── 누적 포인트 클라우드 생성 (프레임 48~50) ─────────────────────────────
if accumulated_points:
    all_points = np.vstack(accumulated_points)
    all_colors = np.vstack(accumulated_colors)
    accumulated_pcd = o3d.geometry.PointCloud()
    accumulated_pcd.points = o3d.utility.Vector3dVector(all_points)
    accumulated_pcd.colors = o3d.utility.Vector3dVector(all_colors)
else:
    accumulated_pcd = None

# ───────────────────── 최종 시각화 ─────────────────────────────
geometries = []
if accumulated_pcd is not None:
    geometries.append(accumulated_pcd)
for bbox in actual_bboxes:
    geometries.append(bbox)
for bbox in predicted_bboxes:
    geometries.append(bbox)

o3d.visualization.draw_geometries(geometries)
